In [ ]:
"""
Purpose: To get the versions of the cells in the current working proofread version
to compare to the older August cells for validation

"""

In [5]:
from os import sys
sys.path.append("/meshAfterParty")
import datajoint as dj
import datajoint_utils as du

minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 06:36:52,818 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 06:36:53,024 - settings - Setting enable_python_native_blobs to True


In [6]:
m65 = minnie

# Getting the Segments to Nucleus Id

In [7]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [ ]:
aug_1_seg_ids,aug_1_nucl_ids = final_nucs.fetch("segment_id","nucleus_id")

# Getting the new Segment Id (from v7)

In [8]:
from annotationframeworkclient import FrameworkClient
from IPython import html, display

/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [9]:
client = FrameworkClient('minnie65_phase3_v1')

In [11]:
import pandas as pd

In [12]:
from annotationframeworkclient import FrameworkClient
client = FrameworkClient('minnie65_phase3_v1')
client.materialize.get_tables()
client.materialize.version
nuc_table_v7 = client.materialize.query_table('nucleus_detection_v0').rename(columns={'id': 'nucleus_id'})
proofread_nucs_df = pd.DataFrame(proofread_nucs.fetch())
final_df = pd.merge(nuc_table_v7, proofread_nucs_df)[['nucleus_id', 'pt_root_id', 'segment_id']].rename(columns={'pt_root_id':'new_seg_id', 'segment_id':'old_seg_id'})
final_df

,nucleus_id,new_seg_id,old_seg_id
0,516758,864691135467717772,864691136537220258
1,517900,864691136811865203,864691136311785789
2,518848,864691134884807418,864691135785277636
3,518853,864691135771790587,864691135866394390
4,518898,864691135583884664,864691136333787571
...,...,...,...
68,493569,864691135660569584,864691135510455760
69,493806,864691136023980601,864691135373402824
70,493885,864691135850577863,864691136752360558
71,493968,864691135117980637,864691135974454383
